# Récupération des liens Etudes d'impacts & Avis de Projet-Environnement

In [1]:
#bibliothèques nécessaires
import pandas as pd 
import re
import datetime
import requests

Il faut indiquer les différents chemins pour :
- indiquer l'emplacement de téléchargement (*cheminEtudeAvis*)
- indiquer le CSV contenant les liens (*cheminLiens*)
- indiquer le CSV contenant les informations des documents déjà traités (*cheminInfos*)
<br/>
<br/>

In [2]:
cheminEtudeAvis = "/Volumes/MIG-1/etude_avis/"
cheminLiens = ""
cheminInfos = "/Volumes/MIG-1/etude_avis/"

In [3]:
#Récupération des liens et des documents déjà traités
liensEtudeAvis = pd.read_csv(cheminLiens+"etude_avis_geo.csv")
dataEtudeAvis = pd.read_csv(cheminInfos+"infos_etude_avis.csv", delimiter=";")
idDone = list(dataEtudeAvis['id'])

In [4]:
liensEtudeAvis.head(5)

,etude_impact,avis_ae,departement,lng,lat
0,http://documents.projets-environnement.gouv.fr...,http://documents.projets-environnement.gouv.fr...,38,6.060826,45.074916
1,http://documents.projets-environnement.gouv.fr...,http://documents.projets-environnement.gouv.fr...,974,55.743148,-21.122115
2,http://documents.projets-environnement.gouv.fr...,http://documents.projets-environnement.gouv.fr...,973,-53.546190,5.506384
3,http://documents.projets-environnement.gouv.fr...,http://documents.projets-environnement.gouv.fr...,31,0.508394,43.168064
4,http://documents.projets-environnement.gouv.fr...,http://documents.projets-environnement.gouv.fr...,57,6.683017,49.169215


In [5]:
dataEtudeAvis.head(5)

,id,etude_impact,avis_ae,departement,longitude,latitude
0,172174,172174_FEI.pdf,172174_AAE.pdf,38,6.060826,45.074916
1,90468,90468_FEI.pdf,90468_AAE.pdf,974,55.743148,-21.122115
2,57536,57536_FEI.pdf,57536_AAE.pdf,973,-53.546190,5.506384
3,189926,189926_FEI.pdf,189926_AAE.pdf,31,0.508394,43.168064
4,770502,770502_FEI.zip,770502_AAE.zip,57,6.683017,49.169215


La cellule qui suit (script principal) peut être interrompu à n'importe quel moment et repris en relançant celle-ci.<br/>
*Exception: pendant la phase de sauvegarde, il ne faut pas interrompre le script*

In [23]:
dataEtudeAvis = pd.read_csv(cheminInfos+"infos_etude_avis.csv", delimiter=";") # permet de relancer la cellule en cas d'interruption sans relancer les cellules précédentes.
idDone = list(dataEtudeAvis['id'])

#TO DELETE (car les fichiers sont en .7z et ne veulent pas se télécharger, même à la main)
idDone.append(93055)
idDone.append(972596)
idDone.append(156605)
#FIN To DELETE

cpt = 0 #compteur pour limité le nombre de fichier à récupérer
maxi = 9999
for index, line in liensEtudeAvis.iterrows():
    
    # Vérification limite
    if(cpt > maxi):
        break
    cpt += 1
    
    
    #récupération de l'ID du fichier
    tmp = re.search('/([^/_]+)_(?:[^/_]+)\.(\w+)$', line['etude_impact'])
    idDoc = str(tmp[1])
    extension_EI = str(tmp[2])
    
    
    if(int(idDoc) not in idDone and extension_EI!='7z'):#Si le document n'a pas déjà été traité
        print("ITERATION "+str(cpt))
        print('extension :', extension_EI)
    
        #################################
        #récupération de l'etude d'impact
        #################################
        
        now = datetime.datetime.now()
        print("   Telechargement etude d'impact... depuis ", now.hour, "h", now.minute)
        page = requests.get(line['etude_impact'])
        #sauvegarde du fichier
        name_file_EI = idDoc+"_FEI."+extension_EI
        etudeImpact = page.content
        

        ###############################
        #récupération de l'avis de l'AE
        ###############################
        
        now = datetime.datetime.now()
        print("   Telechargement avis... depuis ", now.hour, "h", now.minute)
        if(str(line['avis_ae'])!='nan'):
            page = requests.get(line['avis_ae'])
            #sauvegarde du fichier
            tmp = re.search('/([^/_]+)_(?:[^/_]+)\.(\w+)$', line['avis_ae'])
            extension_AE = str(tmp[2])
            name_file_AE = idDoc+"_AAE."+extension_AE
            avis = page.content
            
        else:
            name_file_AE = 'sans_avis'

            
        ############################################
        #sauvegarde informations et noms de fichiers
        ############################################
        print("   Sauvegarde des données (ne pas interrompre)... depuis ", now.hour, "h", now.minute)
        
        #sauvegarde etude impact
        file = open(cheminEtudeAvis+name_file_EI,'wb')
        file.write(etudeImpact)
        file.close()
        
        #sauvegarde avis
        if(str(line['avis_ae'])!='nan'):
            file = open(cheminEtudeAvis+name_file_AE,'wb')
            file.write(avis)
            file.close()
            
        dataCSV = idDoc+";"+name_file_EI+";"+name_file_AE+";"+str(line['departement'])+";"+str(line['lng'])+";"+str(line['lat'])+"\n"
        file = open(cheminInfos+'infos_etude_avis.csv','a')
        file.write(dataCSV)
        file.close()
print("END")

ITERATION 582
extension : pdf
   Telechargement etude d'impact... depuis  18 h 1
   Telechargement avis... depuis  18 h 2
   Sauvegarde des données (ne pas interrompre)... depuis  18 h 2
ITERATION 583
extension : zip
   Telechargement etude d'impact... depuis  18 h 2
   Telechargement avis... depuis  18 h 2
   Sauvegarde des données (ne pas interrompre)... depuis  18 h 2
ITERATION 584
extension : pdf
   Telechargement etude d'impact... depuis  18 h 2
   Telechargement avis... depuis  18 h 4
   Sauvegarde des données (ne pas interrompre)... depuis  18 h 4
ITERATION 585
extension : pdf
   Telechargement etude d'impact... depuis  18 h 4
   Telechargement avis... depuis  18 h 4
   Sauvegarde des données (ne pas interrompre)... depuis  18 h 4
ITERATION 586
extension : pdf
   Telechargement etude d'impact... depuis  18 h 4
   Telechargement avis... depuis  18 h 6
   Sauvegarde des données (ne pas interrompre)... depuis  18 h 6
ITERATION 587
extension : pdf
   Telechargement etude d'impact...

Permet d'afficher simplement les liens de l'étude et de l'ais au cas où le script n'arrievrait pas à télécharger le document

In [16]:
print(line['etude_impact'])

http://documents.projets-environnement.gouv.fr/2019/02/26/100689/100689_FEI.pdf


In [17]:
print(line['avis_ae'])

http://documents.projets-environnement.gouv.fr/2019/02/26/100689/100689_AAE.pdf
